In [1]:
import uuid
from hhutil.io import fmt_path, mv, eglob, rm, Path

def swap_fp(fp1, fp2):
    fp1, fp2 = fmt_path(fp1), fmt_path(fp2)
    if not fp1.exists() and not fp2.exists():
        return
    elif not fp1.exists():
        fp1, fp2 = fp2, fp1
    temp_fp = fp1.parent / uuid.uuid4().hex
    mv(fp1, temp_fp)
    if fp2.exists():
        mv(fp2, fp1)
    mv(temp_fp, fp2)
    

class Sheet:
    MAX_ROWS = 200

    def __init__(self, root) -> None:
        super().__init__()
        root = Path(root)
        if not root.is_absolute():
            root = Path(__file__).parent.absolute().parent / root
        assert root.exists()
        self.root = root
        self.code_dir = root / "code"
        self.log_dir = root / "log"

    def swap(self, row1, row2):
        swap_fp(self.get_code(row1), self.get_code(row2))

        log_files1 = self.get_logs(row1)
        log_files2 = self.get_logs(row2)
        for i in range(1, max(len(log_files1), len(log_files2)) + 1):
            swap_fp(self.get_log(row1, i),self.get_log(row2, i))

    def get_log(self, row, seq):
# if CIFAR, ignore these codes
#         if seq == 1:
#             log = self.log_dir / f"{row}.log"
#             return log
        return self.log_dir / f"{row}-{seq}.log"

    def get_logs(self, row):
        results = list(eglob(self.log_dir, f"{row}-*.log"))
        log = self.log_dir / f"{row}.log"
        if log.exists():
            results.append(log)
        return results

    def get_code(self, row):
        return self.code_dir / f"{row}.py"

    def log_exist(self, row):
        return len(self.get_logs(row)) != 0
    
    def code_exist(self, row):
        return self.get_code(row).exists()

    def exist(self, row):
        return self.code_exist(row) or self.log_exist(row)

    def shift_down(self, row):
        assert self.exist(row) and not self.exist(row + 1)
        self.swap(row, row + 1)

    def shift_up(self, row):
        assert self.exist(row) and not self.exist(row - 1)
        self.swap(row, row - 1)

    def insert(self, row):
        last_row = self.find_last_row()
        for i in reversed(range(row, last_row + 1)):
            self.shift_down(i)

    def find_last_row(self):
        i = self.MAX_ROWS
        while not self.exist(i):
            i -= 1
        return i
        
    def delete(self, row):
        rm(self.get_code(row))
        for f in self.get_logs(row):
            rm(f)
        last_row = self.find_last_row()
        for i in range(row + 1, last_row + 1):
            self.shift_up(i)

In [2]:
sheet = Sheet('C:/libraries/experiments/CIFAR10-TensorFlow-Yang/')

In [3]:
sheet.swap(7, 8)

In [5]:
sheet.swap(12, 13)